In [70]:
import os
%pwd

'D:\\DS 01'

In [71]:
os.chdir("D:\\DS 01")

In [72]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path:Path  
    model_name: str
    ## Elastic Net Model
    alpha: float
    l1_ratio: float
    target_column: str

In [73]:
from src.DS01.constants import *
from src.DS01.utils.common import read_yaml,create_directories

In [74]:
class ConfigurationManager:
    def __init__(self,
                config_filepath=CONFIG_FILE_PATH,
                params_filepath=PARAMS_FILE_PATH,
                schema_filepath=SCHEMA_FILE_PATH):

        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig: 
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])


        model_trainer_config= ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
            )
        return model_trainer_config           


In [75]:
import os
import pandas as pd
from src.DS01 import logger
from sklearn.linear_model import ElasticNet
import joblib

In [76]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config=config
    
    def train(self):

        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)


        train_x=train_data.drop(columns=[self.config.target_column],axis=1)      
        train_y=train_data[self.config.target_column]

        test_x=test_data.drop(columns=[self.config.target_column],axis=1)      
        test_y=test_data[self.config.target_column]


        lr=ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        lr.fit(train_x,train_y)

        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))
    
   

In [77]:
try:
    config=ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer=ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-02-16 23:17:11,058: INFO: common: yaml file :config\config.yaml loaded sucessfully]
[2025-02-16 23:17:11,062: INFO: common: yaml file :params.yaml loaded sucessfully]
[2025-02-16 23:17:11,068: INFO: common: yaml file :schema.yaml loaded sucessfully]
[2025-02-16 23:17:11,071: INFO: common: Directory created at artifacts]
[2025-02-16 23:17:11,073: INFO: common: Directory created at artifacts/model_trainer]
